# DX 704 Week 6 Project

This project will develop a treatment plan for a fictious illness "Twizzleflu".
Twizzleflu is a mild illness caused by a virus.
The main symptoms are a mild fever, fidgeting, and kicking the blankets off the bed or couch.
Mild dehydration has also been reported in more severe cases.
These symptoms typically last 1-2 weeks without treatment.
Word on the internet says that Twizzleflu can be cured faster by drinking copious orange juice, but this has not been supported by evidence so far.
You will be provided with a theoretical model of Twizzleflu modeled as a Markov decision process.
Based on the model, you will compute optimal treatment plans to optimize different criteria, and compare patient discomfort with the different plans.

The full project description, a template notebook, and raw data are available on GitHub: [Project 6 Materials](https://github.com/bu-cds-dx704/dx704-project-06).

We will model Twizzleflu as a Markov decision process.
The model transition probabilities are provided in the file "twizzleflu-transitions.tsv" and the expected rewards are in "twizzleflu-rewards.tsv".
The goal for Twizzleflu is to minimize the expected discomfort of the patient which is expressed as negative rewards in the file.

## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Evaluate a Do Nothing Plan

One of the treatment actions is to do nothing.
Calculate the expected discomfort (not rewards) of a policy that always does nothing.

Hint: for this value calculation and later ones, use value iteration.
The analytical solution has difficulties in practice when there is no discount factor.

In [167]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np

transitions = pd.read_csv("twizzleflu-transitions.tsv", sep="\t")
rewards = pd.read_csv("twizzleflu-rewards.tsv", sep="\t")

def compute_qT_once(R, P, gamma, v):
    return R + gamma * (P @ v)

def iterate_values_once(R, P, gamma, v):
    return np.max(compute_qT_once(R, P, gamma, v), axis=0)

def value_iteration(R, P, gamma, max_iterations=1000, tolerance=1e-12):
    v_old = np.zeros(R.shape[-1], dtype=float)
    for _ in range(max_iterations):
        v_new = iterate_values_once(R, P, gamma, v_old)
        if np.max(np.abs(v_new - v_old)) < tolerance:
            return v_new
        v_old = v_new
    return v_old

In [168]:

P_df = transitions.query('action == "do-nothing"')[["state","next_state","probability"]]
R_df = rewards.query('action == "do-nothing"')[["state","reward"]]

states = sorted(set(P_df["state"]).union(P_df["next_state"]))
states = [s for s in states if s != "recovered"] + ["recovered"]

M = (P_df.pivot_table(index="state", columns="next_state",
                      values="probability", aggfunc="sum")
        .reindex(index=states, columns=states, fill_value=0.0)
        .fillna(0.0))

r_vec = (R_df.set_index("state")["reward"]
           .reindex(states).fillna(0.0))

P = M.to_numpy()[None, :, :]
R = r_vec.to_numpy()[None, :]

gamma = 1.0
V = value_iteration(R, P, gamma)
if "recovered" in states:
    V[states.index("recovered")] = 0.0

out_1 = pd.DataFrame({"state": states, "expected_discomfort": -V})
out_1.to_csv("do-nothing-discomfort.tsv", sep="\t", index=False)
out_1


,state,expected_discomfort
0,exposed-1,3.413333
1,exposed-2,4.266667
2,exposed-3,5.333333
3,symptoms-1,6.666667
4,symptoms-2,5.000000
5,symptoms-3,1.666667
6,recovered,-0.000000


Save the expected discomfort by state to a file "do-nothing-discomfort.tsv" with columns state and expected_discomfort.

Submit "do-nothing-discomfort.tsv" in Gradescope.

## Part 2: Compute an Optimal Treatment Plan

Compute an optimal treatment plan for Twizzleflu.
It should minimize the expected discomfort (maximize the rewards).

In [169]:
# YOUR CHANGES HERE

def iterative_policy_evaluation(R, P, gamma, pi, max_iterations=100, tolerance=0.001):
    # factor out action choices using policy.

    # deterministic version
    n = R.shape[-1]
    R_pi = R[pi, np.arange(n)]
    P_pi = P[pi, np.arange(n),:]

    # reshape to one dummy action to reuse previous example code
    R_pi = R_pi.reshape(1, *R_pi.shape)
    P_pi = P_pi.reshape(1, *P_pi.shape)

    # initial approximation v_0
    v_old = np.zeros(R.shape[-1])

    for i in range(max_iterations):
        # compute v_{i+1}
        v_new = iterate_values_once(R_pi, P_pi, gamma, v_old)

        # check if values did not change much
        if np.max(np.abs(v_new - v_old)) < tolerance:
            return v_new

        v_old = v_new

    # return v_{max_iterations}
    return v_old

def policy_iteration_baseline(R, P, gamma, max_iterations=100, tolerance=0.001):
    pi_old = np.zeros(R.shape[-1], dtype="int64")
    v_old = iterative_policy_evaluation(R, P, gamma, pi_old)

    for i in range(max_iterations):
        # compute new policy
        pi_new = np.argmax(compute_qT_once(R, P, gamma, v_old), axis=0)
        if np.array_equal(pi_new, pi_old):
            return pi_new, v_old

        # compute new values
        v_new = iterative_policy_evaluation(R, P, gamma, pi_new)
        if np.max(np.abs(v_new - v_old)) < tolerance:
            return pi_new, v_new

        pi_old = pi_new
        v_old = v_new

    return pi_old, v_old

In [170]:

A = sorted(transitions["action"].unique())
S = sorted(set(transitions["state"]).union(transitions["next_state"]))


P = (transitions
     .pivot_table(index=["action","state"], columns="next_state",
                  values="probability", aggfunc="sum")
     .reindex(pd.MultiIndex.from_product([A, S], names=["action","state"]))
     .reindex(columns=S, fill_value=0.0)
     .fillna(0.0)
     .to_numpy()
     .reshape(len(A), len(S), len(S)))

R = (rewards
     .pivot_table(index=["action","state"], values="reward", aggfunc="sum")
     .reindex(pd.MultiIndex.from_product([A, S], names=["action","state"]))
     .fillna(0.0)
     .to_numpy()
     .reshape(len(A), len(S)))

gamma = 1.0
pi_star_idx, v_star = policy_iteration_baseline(R, P, gamma)



Save the optimal actions for each state to a file "minimum-discomfort-actions.tsv" with columns state and action.

In [171]:
# YOUR CHANGES HERE

policy_2 = pd.DataFrame({
    "state": S,
    "action": [A[i] for i in pi_star_idx]
})
policy_2.loc[policy_2["state"] == "recovered", "action"] = "do-nothing"

policy_2.to_csv("minimum-discomfort-actions.tsv", sep="\t", index=False)
policy_2


,state,action
0,exposed-1,sleep-8
1,exposed-2,sleep-8
2,exposed-3,sleep-8
3,recovered,do-nothing
4,symptoms-1,drink-oj
5,symptoms-2,drink-oj
6,symptoms-3,drink-oj


Submit "minimum-discomfort-actions.tsv" in Gradescope.

## Part 3: Expected Discomfort

Using your previous optimal policy, compute the expected discomfort for each state.

In [172]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd

v_opt = iterative_policy_evaluation(R, P, gamma, pi_star_idx)


Save your results in a file "minimum-discomfort-values.tsv" with columns state and expected_discomfort.

In [173]:
# YOUR CHANGES HERE

out_3 = pd.DataFrame({"state": S, "expected_discomfort": -v_opt})
out_3.to_csv("minimum-discomfort-values.tsv", sep="\t", index=False)
out_3

,state,expected_discomfort
0,exposed-1,0.749165
1,exposed-2,1.498689
2,exposed-3,2.997944
3,recovered,-0.000000
4,symptoms-1,5.996779
5,symptoms-2,4.499580
6,symptoms-3,1.499973


Submit "minimum-discomfort-values.tsv" in Gradescope.

## Part 4: Minimizing Twizzleflu Duration

Modifiy the Markov decision process to minimize the days until the Twizzle flu is over.
To do so, change the reward function to always be -1 if the current state corresponds to being sick and 0 if the current state corresponds to being better.
To be clear, the action does not matter for this reward function.


In [174]:
# YOUR CHANGES HERE

rows = [
    {"action": a, "state": s, "reward": (0.0 if s == "recovered" else -1.0)}
    for a in A for s in S
]

Save your new reward function in a file "duration-rewards.tsv" in the same format as "twizzleflu-rewards.tsv".

In [175]:
# YOUR CHANGES HERE

duration_rewards = pd.DataFrame(rows, columns=["action", "state", "reward"])
duration_rewards.to_csv("duration-rewards.tsv", sep="\t", index=False)
duration_rewards

,action,state,reward
0,do-nothing,exposed-1,-1.0
1,do-nothing,exposed-2,-1.0
2,do-nothing,exposed-3,-1.0
3,do-nothing,recovered,0.0
4,do-nothing,symptoms-1,-1.0
5,do-nothing,symptoms-2,-1.0
6,do-nothing,symptoms-3,-1.0
7,drink-oj,exposed-1,-1.0
8,drink-oj,exposed-2,-1.0
9,drink-oj,exposed-3,-1.0


Submit "duration-rewards.tsv" in Gradescope.

## Part 5: Optimize for Shorter Twizzleflu

Compute an optimal policy to minimize the duration of Twizzleflu.

In [176]:
# YOUR CHANGES HERE

duration_rewards = pd.read_csv("duration-rewards.tsv", sep="\t")

R_5 = (duration_rewards
     .pivot_table(index=["action","state"], values="reward", aggfunc="sum")
     .reindex(pd.MultiIndex.from_product([A, S], names=["action","state"]))
     .fillna(0.0)
     .to_numpy()
     .reshape(len(A), len(S)))

pi_star_idx, v_star = policy_iteration_baseline(R_5, P, gamma)

policy_5 = pd.DataFrame({"state": S, "action": [A[i] for i in pi_star_idx]})
policy_5.loc[policy_5["state"] == "recovered", "action"] = "do-nothing"



Save the optimal actions for each state to a file "minimum-duration-actions.tsv" with columns state and action.

In [177]:
# YOUR CHANGES HERE

policy_5.to_csv("minimum-duration-actions.tsv", sep="\t", index=False)
policy_5

,state,action
0,exposed-1,sleep-8
1,exposed-2,sleep-8
2,exposed-3,sleep-8
3,recovered,do-nothing
4,symptoms-1,do-nothing
5,symptoms-2,do-nothing
6,symptoms-3,do-nothing


Submit "minimum-duration-actions.tsv" in Gradescope.

## Part 6: Shorter Twizzleflu?

Compute the expected number of days sick for each state to a file.

In [178]:
# YOUR CHANGES HERE

pi_star_idx, v_star = policy_iteration_baseline(R_5, P, gamma)

Save the expected sick days for each state to a file "minimum-duration-days.tsv" with columns state and expected_sick_days.

In [179]:
# YOUR CHANGES HERE

pd.DataFrame({"state": S, "expected_sick_days": -v_star}) \
  .to_csv("minimum-duration-days.tsv", sep="\t", index=False)

Submit "minimum-duration-days.tsv" in Gradescope.

## Part 7: Speed vs Pampering

Compute the expected discomfort using the policy to minimize days sick, and compare the results to the expected discomfort when optimizing to minimize discomfort.

In [180]:
# YOUR CHANGES HERE

R_orig = (rewards
          .pivot_table(index=["action","state"], values="reward", aggfunc="sum")
          .reindex(pd.MultiIndex.from_product([A, S], names=["action","state"]))
          .fillna(0.0).to_numpy().reshape(len(A), len(S)))

duration_rewards = pd.read_csv("duration-rewards.tsv", sep="\t")
R_dur = (duration_rewards
         .pivot_table(index=["action","state"], values="reward", aggfunc="sum")
         .reindex(pd.MultiIndex.from_product([A, S], names=["action","state"]))
         .fillna(0.0).to_numpy().reshape(len(A), len(S)))

pi_speed, _      = policy_iteration_baseline(R_dur,  P, gamma)
pi_comfort, v_c  = policy_iteration_baseline(R_orig, P, gamma)

v_speed_eval   = iterative_policy_evaluation(R_orig, P, gamma, pi_speed)
v_comfort_eval = iterative_policy_evaluation(R_orig, P, gamma, pi_comfort)

speed_discomfort    = -v_speed_eval.copy()
minimize_discomfort = -v_comfort_eval.copy()

if "recovered" in S:
    speed_discomfort[S.index("recovered")] = 0.0
    minimize_discomfort[S.index("recovered")] = 0.0

Save the results to a file "policy-comparison.tsv" with columns state, speed_discomfort, and minimize_discomfort.

In [181]:
# YOUR CHANGES HERE

out_7 = pd.DataFrame({
    "state": S,
    "speed_discomfort":    speed_discomfort,
    "minimize_discomfort": minimize_discomfort
})
out_7.to_csv("policy-comparison.tsv", sep="\t", index=False)
out_7

,state,speed_discomfort,minimize_discomfort
0,exposed-1,0.832556,0.749165
1,exposed-2,1.665517,1.498689
2,exposed-3,3.331634,2.997944
3,recovered,0.000000,0.000000
4,symptoms-1,6.664159,5.996779
5,symptoms-2,4.999692,4.499580
6,symptoms-3,1.666648,1.499973


Submit "policy-comparison.tsv" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.